In [1]:
# ============================================
#  Laboratorio VDT
#  Ordenado por secciones: funciones -> ejemplos
# ============================================

In [2]:
# --------------------------------------------
# Punto A. Montos únicos: VP y VF
# --------------------------------------------
def vp(vf, r, n):
    """Valor Presente de un monto único: VP = VF / (1+r)^n"""
    return vf / ((1 + r) ** n)

def vf(vp0, r, n):
    """Valor Futuro de un monto único: VF = VP0 * (1+r)^n"""
    return vp0 * ((1 + r) ** n)

In [3]:
help(vp)

Help on function vp in module __main__:

vp(vf, r, n)
    Valor Presente de un monto único: VP = VF / (1+r)^n



In [4]:
# --- Demostración A ---
print("=== Punto A: VP y VF (monto único) ===")
r_anual = 0.12
n = 5
vf_obj = 50000.0
vp_5 = vp(vf_obj, r_anual, n)
print(f"VP de ${vf_obj:,.0f} en {n} años a r={r_anual:.1%} -> ${vp_5:,.2f}")

vp0 = 30000.0
vf_5 = vf(vp0, r_anual, n)
print(f"VF de ${vp0:,.0f} en {n} años a r={r_anual:.1%} -> ${vf_5:,.2f}")
print()


=== Punto A: VP y VF (monto único) ===
VP de $50,000 en 5 años a r=12.0% -> $28,371.34
VF de $30,000 en 5 años a r=12.0% -> $52,870.25



In [5]:
# --------------------------------------------
# Punto B. Anualidades (pagos iguales al final)
# --------------------------------------------
def vp_anualidad(pmt, r, n):
    """VP de una anualidad ordinaria (pagos al final)."""
    if r == 0:
        return pmt * n
    return pmt * (1 - (1 + r) ** (-n)) / r

def vf_anualidad(pmt, r, n):
    """VF de una anualidad ordinaria (pagos al final)."""
    if r == 0:
        return pmt * n
    return pmt * ((1 + r) ** n - 1) / r

def pmt_desde_vp(VP_obj, r, n):
    """Pago periódico requerido para alcanzar un VP objetivo en n periodos."""
    if r == 0:
        return VP_obj / n
    return VP_obj * r / (1 - (1 + r) ** (-n))

In [6]:
# --- Demostración B ---
print("=== Punto B: Anualidades (VP, VF y PMT) ===")
pmt = 10000.0
n = 5
vp_an = vp_anualidad(pmt, r_anual, n)
vf_an = vf_anualidad(pmt, r_anual, n)
print(f"VP de una anualidad de ${pmt:,.0f} por {n} años a {r_anual:.1%}: ${vp_an:,.2f}")
print(f"VF de la misma anualidad: ${vf_an:,.2f}")

VP_obj = 100000.0
pmt_req = pmt_desde_vp(VP_obj, r_anual, n)
print(f"PMT para lograr VP=${VP_obj:,.0f} en {n} años a {r_anual:.1%}: ${pmt_req:,.2f}")
print()

=== Punto B: Anualidades (VP, VF y PMT) ===
VP de una anualidad de $10,000 por 5 años a 12.0%: $36,047.76
VF de la misma anualidad: $63,528.47
PMT para lograr VP=$100,000 en 5 años a 12.0%: $27,740.97



In [7]:
# --------------------------------------------
# Punto C. Series generales: VAN y TIR
# --------------------------------------------
def van(r, cashflows):
    """
    Valor Actual Neto de una serie [CF0, CF1, ..., CFn].
    CF0 suele ser negativo (inversión inicial).
    """
    total = 0.0
    for t, cf in enumerate(cashflows):
        total += cf / ((1 + r) ** t)
    return total

def irr(cashflows, guess=0.10, tol=1e-8, maxiter=200):
    """
    TIR mediante Newton-Raphson.
    Devuelve float('nan') si no converge.
    """
    def f(rate):
        return van(rate, cashflows)
    r = guess
    for _ in range(maxiter):
        f0 = f(r)
        # derivada numérica por diferencia hacia adelante
        h = 1e-6
        f1 = (f(r + h) - f0) / h
        if abs(f1) < 1e-14:
            break
        r_next = r - f0 / f1
        if abs(r_next - r) < tol:
            return r_next
        r = r_next
    return float('nan')


In [8]:
# --- Demostración C ---
print("=== Punto C: VAN y TIR ===")
# Proyecto ejemplo: inversión inicial y cobros anuales
cashflows = [-100000.0, 30000.0, 35000.0, 40000.0, 45000.0, 50000.0]
r_desc = 0.12
van_val = van(r_desc, cashflows)
tir_val = irr(cashflows)
print(f"VAN a r={r_desc:.1%}: ${van_val:,.2f} -> {'ACEPTAR' if van_val>0 else 'RECHAZAR'}")
print(f"TIR aproximada del proyecto: {tir_val*100:.2f}%")
print()

=== Punto C: VAN y TIR ===
VAN a r=12.0%: $40,128.37 -> ACEPTAR
TIR aproximada del proyecto: 25.75%



In [9]:
# --------------------------------------------
# Punto D. Tasas efectivas y tasa real (Fisher)
# --------------------------------------------
def tasa_efectiva(nominal, m=1):
    """
    Convierte una tasa nominal j capitalizable m veces al año
    en la tasa efectiva anual equivalente: (1 + j/m)^m - 1
    """
    return (1 + nominal / m) ** m - 1

def fisher_efectiva(i_nom, pi):
    """
    Tasa real exacta: (1 + i_nom) / (1 + pi) - 1
    i_nom: tasa nominal (por ejemplo, interés de mercado)
    pi: inflación esperada del periodo
    """
    return (1 + i_nom) / (1 + pi) - 1


In [10]:
# --- Demostración D ---
print("=== Punto D: Tasas efectivas y Fisher ===")
j_nom = 0.18   # 18% nominal
m = 12         # capitalización mensual
i_eff = tasa_efectiva(j_nom, m)
pi = 0.06      # 6% inflación
i_real = fisher_efectiva(i_eff, pi)
print(f"Tasa efectiva anual equivalente de {j_nom:.1%} nominal con m={m}: {i_eff:.2%}")
print(f"Tasa real (Fisher) con inflación {pi:.1%}: {i_real:.2%}")
print()

=== Punto D: Tasas efectivas y Fisher ===
Tasa efectiva anual equivalente de 18.0% nominal con m=12: 19.56%
Tasa real (Fisher) con inflación 6.0%: 12.79%



## Informe automatico

In [11]:
# 1) Instalar dependencias (solo 1 vez por sesión de Colab)
!pip -q install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 4.9 MB/s eta 0:00:00


In [12]:
# Librerias necesarias.
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml.ns import qn
from datetime import datetime
import matplotlib.pyplot as plt
import os

In [13]:
# Funciones necesarias y auxiliares.

def add_heading(doc, text, level=1):
    h = doc.add_heading(text, level=level)
    return h

def add_paragraph(doc, text, bold=False, italic=False):
    p = doc.add_paragraph()
    run = p.add_run(text)
    run.bold = bold
    run.italic = italic
    return p

def add_key_value_table(doc, data_dict, col1="Métrica", col2="Valor"):
    table = doc.add_table(rows=1, cols=2)
    hdr_cells = table.rows[0].cells
    hdr_cells[0].text = col1
    hdr_cells[1].text = col2
    for k, v in data_dict.items():
        row_cells = table.add_row().cells
        row_cells[0].text = str(k)
        row_cells[1].text = str(v)
    table.style = "Light List"
    doc.add_paragraph("")  # espacio

def formato_moneda(x):
    try:
        return f"${float(x):,.2f}"
    except:
        return str(x)

In [14]:
# Funciones para graficos.

def plot_perfil_van(cashflows, r_min=0.0, r_max=0.4, pasos=200, TIR=None):
    tasas = [r_min + i*(r_max-r_min)/(pasos-1) for i in range(pasos)]
    def van(rate, cfs):
        return sum(cf / ((1+rate)**t) for t, cf in enumerate(cfs))
    vans = [van(r, cashflows) for r in tasas]
    fig = plt.figure()
    plt.plot([t*100 for t in tasas], vans)
    plt.axhline(0, color="k", lw=1)
    if TIR is not None and TIR == TIR:  # no NaN
        plt.axvline(TIR*100, color="r", ls="--", label=f"TIR ≈ {TIR*100:.2f}%")
        plt.legend()
    plt.title("Perfil del VAN")
    plt.xlabel("Tasa de descuento (%)")
    plt.ylabel("VAN")
    plt.grid(True)
    return fig

def plot_vp_acumulado(cashflows, r):
    vp = [cf / ((1+r)**t) for t, cf in enumerate(cashflows)]
    vp_acum = []
    s = 0.0
    for v in vp:
        s += v
        vp_acum.append(s)
    fig = plt.figure()
    plt.plot(range(len(vp_acum)), vp_acum, marker="o")
    plt.axhline(0, color="k", lw=1)
    plt.title(f"Valor Presente Acumulado (r={r:.1%})")
    plt.xlabel("Periodo")
    plt.ylabel("VP acumulado")
    plt.grid(True)
    return fig

def save_fig(fig, path_png):
    fig.savefig(path_png, dpi=150, bbox_inches="tight")
    plt.close(fig)

In [15]:
# ---------- Armar el reporte ----------
# Asegúrate de tener definidas estas variables en tu notebook:
# r_anual, inflacion, vp_5, vf_5, vp_an, vp_an, pmt_req, VAN_base, TIR, cashflows

# Si alguna no existe, ajusta aquí (o comenta lo que no tengas):
parametros = {
    "Tasa de descuento anual (r)": f"{r_anual:.2%}",
    "Inflación esperada (π)": f"{pi:.2%}",
}

resultados = {
    "VP de VF=50,000 a 5 años": formato_moneda(vp_5),
    "VF de VP0=30,000 a 5 años": formato_moneda(vf_5),
    "VP Anualidad (PMT=10,000, n=5)": formato_moneda(vp_an),
    "VF Anualidad (PMT=10,000, n=5)": formato_moneda(vf_an),
    "PMT para VP objetivo=100,000 (n=5)": formato_moneda(pmt_req),
    f"VAN proyecto a r={r_anual:.0%}": formato_moneda(van_val),
    "TIR del proyecto": f"{tir_val*100:.2f}%",
}

In [16]:
# Crear carpeta y nombre de archivo con timestamp
os.makedirs("reports_vdt", exist_ok=True)
stamp = datetime.now().strftime("%Y%m%d_%H%M")
docx_path = f"reports_vdt/Reporte_VDT_{stamp}.docx"

In [17]:
# Documento
doc = Document()
title = doc.add_heading("Reporte — Valor del Dinero en el Tiempo", level=0)
title.alignment = WD_ALIGN_PARAGRAPH.CENTER
add_paragraph(doc, f"Generado automáticamente: {datetime.now().strftime('%Y-%m-%d %H:%M')}", italic=True)

In [18]:
# Sección 1: Parámetros
add_heading(doc, "1. Parámetros usados", level=1)
add_key_value_table(doc, parametros, "Parámetro", "Valor")

# Sección 2: Resultados clave
add_heading(doc, "2. Resultados clave", level=1)
add_key_value_table(doc, resultados, "Métrica", "Resultado")

# (Opcional) Sección 3: Gráficos
try:
    fig1 = plot_perfil_van(cashflows, r_min=0.0, r_max=0.4, pasos=200, TIR=TIR)
    png1 = f"reports_vdt/perfil_VAN_{stamp}.png"
    save_fig(fig1, png1)

    fig2 = plot_vp_acumulado(cashflows, r=r_anual)
    png2 = f"reports_vdt/vp_acumulado_{stamp}.png"
    save_fig(fig2, png2)

    add_heading(doc, "3. Gráficos", level=1)
    add_paragraph(doc, "3.1 Perfil del VAN")
    doc.add_picture(png1, width=Inches(6))
    add_paragraph(doc, "3.2 Valor Presente Acumulado")
    doc.add_picture(png2, width=Inches(6))
except Exception as e:
    add_paragraph(doc, f"(No se incluyeron gráficos: {e})", italic=True)

# Sección 4: Observaciones
add_heading(doc, "4. Observaciones y notas", level=1)
add_paragraph(
    doc,
    "- VAN > 0 sugiere aceptar el proyecto a la tasa de descuento indicada.\n"
    "- La TIR es la tasa que hace VAN = 0; si hay múltiples cambios de signo en CF, puede haber múltiples TIR.\n"
    "- VP/VF y anualidades se interpretan en el marco del VDT puro; no incorporan riesgo por sí solos.\n"
)

In [19]:
# Guardar
doc.save(docx_path)
print("Reporte DOCX generado en:", os.path.abspath(docx_path))

Reporte DOCX generado en: /content/reports_vdt/Reporte_VDT_20250830_1626.docx
